<a href="https://colab.research.google.com/github/a01137646/MNA_Public/blob/main/MNA_IAyAA_semana_8_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 8**
###**Máquina de Vector Soporte - Support Vector Machine(SVM)**


**Nombres y matrículas de los integrantes del equipo:**

* Gerardo Aaron Castañeda Jaramillo - A01137646
* Dalina Aidee Villa Ocelotl - A01793258
* Yocelin Juárez Arroyo - A00819828
* Victor Hugo Avila Felipe - A01794425


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [1]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [48]:
mypath = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/australian/australian.dat"

data = pd.read_csv(mypath, sep=" ", header=None)

data.columns = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','class']

print(data.shape)
data.head()

(690, 15)


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,class
0,1,22.08,11.46,2,4,4,1.585,0,0,0,1,2,100,1213,0
1,0,22.67,7.00,2,8,4,0.165,0,0,0,0,2,160,1,0
2,0,29.58,1.75,1,4,4,1.250,0,0,0,1,2,280,1,0
3,0,21.67,11.50,1,5,3,0.000,1,1,11,1,2,0,1,1
4,1,20.17,8.17,2,6,4,1.960,1,1,14,0,2,60,159,1


In [37]:
Y = data[['class']]
X = data.drop(labels='class',axis=1)

#**Ejercicio-1.** 

Realiza una partición de los datos con 15% en el conjunto de prueba y el resto para entrenamiento y validación. Usa un random_state=0. 

In [13]:
Xtv, Xtest, ytv, ytest = train_test_split(X, Y, test_size=0.15, stratify=Y, random_state=0)  

print(Xtv.shape, ': dimensión de datos de entrada para entrenamiento y validación')
print(Xtest.shape, ': dimensión de datos de entrada para prueba')  

print(ytv.shape, ': dimensión de variable de salida para entrenamiento y validación')
print(ytest.shape, ': dimensión de variable de salida para prueba')

(586, 14) : dimensión de datos de entrada para entrenamiento y validación
(104, 14) : dimensión de datos de entrada para prueba
(586, 1) : dimensión de variable de salida para entrenamiento y validación
(104, 1) : dimensión de variable de salida para prueba


#**Ejercicio-2.**

Hacer una análisis de los datos y llevar a cabo las transformaciones que consideres adecuadas. Utiliza un Pipeline para evitar el filtrado de información

Observando lo que nos dice la documentación:
* A1:	0,1    CATEGORICAL a,b
* A2:	continuous.
* A3:	continuous.
* A4:	1,2,3         CATEGORICAL p,g,gg
* A5:  1, 2,3,4,5, 6,7,8,9,10,11,12,13,14    CATEGORICAL ff,d,i,k,j,aa,m,c,w, e, q, r,cc, x 
* A6:	 1, 2,3, 4,5,6,7,8,9    CATEGORICAL ff,dd,j,bb,v,n,o,h,z 
* A7:	continuous.
* A8:	1, 0       CATEGORICAL t, f.
* A9: 1, 0	    CATEGORICAL t, f.
* A10:	continuous.
* A11:  1, 0	    CATEGORICAL t, f.
* A12:    1, 2, 3    CATEGORICAL s, g, p 
* A13:	continuous.
* A14:	continuous.
* A15:   1,2 +,-         (class attribute)


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      690 non-null    int64  
 1   A2      690 non-null    float64
 2   A3      690 non-null    float64
 3   A4      690 non-null    int64  
 4   A5      690 non-null    int64  
 5   A6      690 non-null    int64  
 6   A7      690 non-null    float64
 7   A8      690 non-null    int64  
 8   A9      690 non-null    int64  
 9   A10     690 non-null    int64  
 10  A11     690 non-null    int64  
 11  A12     690 non-null    int64  
 12  A13     690 non-null    int64  
 13  A14     690 non-null    int64  
 14  class   690 non-null    int64  
dtypes: float64(3), int64(12)
memory usage: 81.0 KB


In [23]:
data.isnull().values.any()

False

Definimos variables por sus tipos en base a la documentación:

In [20]:
#definimos variables numéricas
num_nom = ['A2','A3','A7','A10','A13','A14']
#definimos variables categóricas
cat_nom = ['A1', 'A4', 'A5', 'A6', 'A8', 'A9', 'A11', 'A12']

Observamos la frecuencia de cada nivel de las variables categóricas:

In [49]:
for col in cat_nom:
  print(data[col].value_counts(),'\n')

1    468
0    222
Name: A1, dtype: int64 

2    525
1    163
3      2
Name: A4, dtype: int64 

8     146
11     78
9      64
3      59
6      54
1      53
4      51
13     41
14     38
7      38
2      30
10     25
5      10
12      3
Name: A5, dtype: int64 

4    408
8    138
5     59
1     57
3      8
9      8
2      6
7      6
Name: A6, dtype: int64 

1    361
0    329
Name: A8, dtype: int64 

0    395
1    295
Name: A9, dtype: int64 

0    374
1    316
Name: A11, dtype: int64 

2    625
1     57
3      8
Name: A12, dtype: int64 



Una función para remplazar valores menores a cierto porcentaje del total de información con otro valor, para así poder agrupar: 

In [50]:
def freq_group(df,n,tres):
  x = df.copy()
  for indx in x.value_counts().index.tolist():
    if (x.value_counts()[indx]/x.value_counts().sum()) < tres:
      x = x.replace(to_replace=indx,value=n)
  return x

Vemos específicamente cada caso, y hacemos sus sustituciones: 

In [53]:
#Para A4, agrupamos el valor menor de la categoria mas baja junto con el de la segunda mas baja, que se encuentra arriba del 20%
for var in ['A4','A12']:
  data[var] = freq_group(data[var],0,0.25)
#Para A5, A6, y A12 agrupamos todos los menores a 5% en un solo grupo
for var in ['A5','A6']:
  data[var] = freq_group(data[var],0,0.05)

Observamos la frecuencia de cada nivel de las variables categóricas:

In [54]:
for col in cat_nom:
  print(data[col].value_counts(),'\n')

1    468
0    222
Name: A1, dtype: int64 

2    525
0    165
Name: A4, dtype: int64 

8     146
11     78
0      68
9      64
3      59
6      54
1      53
4      51
13     41
14     38
7      38
Name: A5, dtype: int64 

4    408
8    138
5     59
1     57
0     28
Name: A6, dtype: int64 

1    361
0    329
Name: A8, dtype: int64 

0    395
1    295
Name: A9, dtype: int64 

0    374
1    316
Name: A11, dtype: int64 

2    625
0     65
Name: A12, dtype: int64 



Observamos que algunos efectivamente, tienen niveles muy bajos. Una regla empírica para esto es que sean al menos 5% de la información. Así que transformamos aquellos que no lo cumplen. 

Verificamos que no existan clases desbalanceadas:

In [14]:
ytv.sum() / ytv.shape[0]

class    0.445392
dtype: float64

In [ ]:
None

#**Ejercicio-3.**

In [ ]:
None

#**Ejercicio-4.**

In [ ]:
None

#**Ejercicio-5.**

In [ ]:
None

#**Ejercicio-6.**

In [ ]:
None

#**Ejercicio-7.**

In [ ]:
None

#**Ejercicio-8.**

In [ ]:
None

#**Ejercicio-9.**

In [ ]:
None

#**Ejercicio-10.**

In [ ]:
None

###**Fin de la Actividad de la semana 8.**